In [51]:
from keras import regularizers
from keras import applications
from keras import utils
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import numpy as np
import pandas as pd
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.cross_validation import train_test_split

In [39]:


nb_train_samples = 1203 ## todo
nb_validation_samples = 466 ## todo
batch_size = 32
epochs = 30

In [40]:
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (75, 75, 3))


In [41]:
train = pd.read_json("train.json")
test = pd.read_json("train.json")
X1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
#X3=np.zeros((1604,75,75))
X_t= np.concatenate([X1[:, :, :, np.newaxis], X2[:, :, :, np.newaxis],((X1+X2)/2)[:, :, :, np.newaxis]], axis=-1)


In [42]:
target_train=train['is_iceberg']
target_train=utils.to_categorical(target_train, num_classes=None)
X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_t, target_train, random_state=1, train_size=0.75)

In [43]:
for layer in model.layers:  # freeze the layers 
    layer.trainable = False

In [52]:
x = model.output
x = Flatten()(x)
x = Dense(512, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
x = Dropout(0.2)(x)
x = Dense(512, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
x = Dropout(0.2)(x)
predictions = Dense(2, activation="sigmoid",kernel_regularizer=regularizers.l2(0.01))(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

mypotim=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model_final.compile(loss='binary_crossentropy',optimizer=mypotim,metrics=['accuracy'])
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 75, 75, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 75, 75, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 37, 37, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 37, 37, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 37, 37, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 18, 18, 128)       0         
__________

/home/andreas/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


the regularization is really necessary not to overfit

In [53]:
def get_callbacks(filepath, patience=3):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]# maybe deactivate checkpoint to be faster, maybe change early stopping
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=3)

In [54]:
model_final.fit(X_train_cv, y_train_cv,
          batch_size=36,
          epochs=10,
          verbose=1,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Train on 1203 samples, validate on 401 samples
Epoch 1/10
1203/1203 [==============================] - 188s - loss: 7.7729 - acc: 0.6912 - val_loss: 3.5403 - val_acc: 0.8167
Epoch 2/10
1203/1203 [==============================] - 187s - loss: 2.3758 - acc: 0.8101 - val_loss: 1.6538 - val_acc: 0.7868
Epoch 3/10
1203/1203 [==============================] - 188s - loss: 1.3110 - acc: 0.8196 - val_loss: 1.0584 - val_acc: 0.8317
Epoch 4/10
1203/1203 [==============================] - 187s - loss: 0.9341 - acc: 0.8321 - val_loss: 0.8229 - val_acc: 0.8279
Epoch 5/10
1203/1203 [==============================] - 188s - loss: 0.7916 - acc: 0.8346 - val_loss: 0.7464 - val_acc: 0.8155
Epoch 6/10
1203/1203 [==============================] - 188s - loss: 0.6535 - acc: 0.8454 - val_loss: 0.6266 - val_acc: 0.8254
Epoch 7/10
1203/1203 [==============================] - 188s - loss: 0.5806 - acc: 0.8558 - val_loss: 0.5816 - val_acc: 0.8529
Epoch 8/10
1203/1203 [==============================] - 189s - l

In [55]:
score = model_final.evaluate(X_valid, y_valid, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

401/401 [==============================] - 47s    
Test loss: 0.553091874146
Test accuracy: 0.824189526779
